In [ ]:
import json
import datetime

In [ ]:
data = {
    'latitude' : 55.642280, 
    'longitude' : 37.568376
}

file_path = "data/testdata/data.json"

# Открываем файл и сохраняем данные в формате JSON
with open(file_path, "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

: 

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
from PIL import Image
import cv2

In [6]:
model = tf.saved_model.load('centernet_hg104_1024x1024_kpts_coco17_tpu-32/saved_model')


In [7]:
for key, value in model.signatures.items():
    print(f"Signature key: {key}")
    print(value.structured_input_signature)
    print(value.structured_outputs)


Signature key: serving_default
((), {'input_tensor': TensorSpec(shape=(1, None, None, 3), dtype=tf.uint8, name='input_tensor')})
{'detection_classes': TensorSpec(shape=(1, 100), dtype=tf.float32, name='detection_classes'), 'detection_keypoint_scores': TensorSpec(shape=(1, 100, 17), dtype=tf.float32, name='detection_keypoint_scores'), 'detection_keypoints': TensorSpec(shape=(1, 100, 17, 2), dtype=tf.float32, name='detection_keypoints'), 'num_detections': TensorSpec(shape=(1,), dtype=tf.float32, name='num_detections'), 'detection_boxes': TensorSpec(shape=(1, 100, 4), dtype=tf.float32, name='detection_boxes'), 'detection_scores': TensorSpec(shape=(1, 100), dtype=tf.float32, name='detection_scores')}


(
    (), 
    {
        'input_tensor': 
            TensorSpec(
                shape=(1, None, None, 3), 
                dtype=tf.uint8, name='input_tensor'
                )
    }   
)
{
    'detection_classes': 
        TensorSpec(
            shape=(1, 100), 
            dtype=tf.float32, 
            name='detection_classes'
            ), 

    'detection_keypoint_scores': 
        TensorSpec(
            shape=(1, 100, 17), 
            dtype=tf.float32, 
            name='detection_keypoint_scores'
            ), 

    'detection_keypoints': 
        TensorSpec(
            shape=(1, 100, 17, 2), 
            dtype=tf.float32, 
            name='detection_keypoints'
            ), 

    'num_detections': 
        TensorSpec(
            shape=(1,), 
            dtype=tf.float32, 
            name='num_detections'
            ), 

    'detection_boxes': 
        TensorSpec(
            shape=(1, 100, 4), 
            dtype=tf.float32, 
            name='detection_boxes'
            ), 

    'detection_scores': 
        TensorSpec(
            shape=(1, 100), 
            dtype=tf.float32, 
            name='detection_scores'
            )
}

In [7]:
image_path = 'data/testdata/photo.jpg'
image = Image.open(image_path)

# Если изображение в формате grayscale, преобразуйте его в RGB
if image.mode != 'RGB':
    image = image.convert('RGB')

# Преобразуем изображение в массив NumPy с типом uint8
image_array = np.array(image, dtype=np.uint8)

# Добавляем измерение для батча
input_tensor = np.expand_dims(image_array, axis=0)

In [8]:
infer = model.signatures['serving_default']

# Выполнение предсказания
output = infer(tf.convert_to_tensor(input_tensor))

2024-11-26 22:15:09.376595: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:933] Skipping loop optimization for Merge node with control input: StatefulPartitionedCall/cond/then/_918/cond/Assert_3/AssertGuard/branch_executed/_1132


In [9]:
detection_classes = output['detection_classes'].numpy()
detection_scores = output['detection_scores'].numpy()
detection_boxes = output['detection_boxes'].numpy()
num_detections = int(output['num_detections'].numpy()[0])

# Вывод результатов для каждого обнаруженного объекта
for i in range(num_detections):
    print(f"Object {i + 1}:")
    print(f" - Class: {detection_classes[0][i]}")
    print(f" - Score: {detection_scores[0][i]}")
    print(f" - Box: {detection_boxes[0][i]}")

Object 1:
 - Class: 1.0
 - Score: 0.9316099286079407
 - Box: [0.25465783 0.         0.9993806  0.9352244 ]
Object 2:
 - Class: 67.0
 - Score: 0.031668879091739655
 - Box: [0.99583155 0.         1.         1.        ]
Object 3:
 - Class: 81.0
 - Score: 0.025358345359563828
 - Box: [0.15378699 0.         0.18544316 0.03452624]
Object 4:
 - Class: 67.0
 - Score: 0.023132704198360443
 - Box: [0.95027494 0.00411908 0.99880314 1.        ]
Object 5:
 - Class: 84.0
 - Score: 0.022984908893704414
 - Box: [0.00089125 0.00030014 0.1744952  0.03751066]
Object 6:
 - Class: 84.0
 - Score: 0.02252296917140484
 - Box: [0.15378699 0.         0.18544316 0.03452624]
Object 7:
 - Class: 72.0
 - Score: 0.02219783514738083
 - Box: [0.00025619 0.00035931 0.16759983 0.03759189]
Object 8:
 - Class: 78.0
 - Score: 0.021573442965745926
 - Box: [0.00025619 0.00035931 0.16759983 0.03759189]
Object 9:
 - Class: 72.0
 - Score: 0.021193960681557655
 - Box: [0.00261135 0.00041733 0.24360989 0.03724681]
Object 10:
 - C

In [ ]:
# 1. Загрузка изображения с использованием Pillow
image_path = 'data/testdata/photo.jpg'
image = Image.open(image_path)

# Преобразование изображения в RGB (если оно не RGB)
if image.mode != 'RGB':
    image = image.convert('RGB')

# Преобразование изображения в массив NumPy с типом uint8
image_np = np.array(image, dtype=np.uint8)

# Преобразование изображения в формат BGR для использования с OpenCV
image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

# Преобразование изображения в массив NumPy с типом uint8 и добавление измерения для батча
input_tensor = np.expand_dims(image_np, axis=0)

# 2. Загрузка модели и выполнение предсказания
infer = model.signatures['serving_default']

# Выполнение предсказания
output = infer(tf.convert_to_tensor(input_tensor))

# 3. Извлечение результатов модели
# Извлечение выходных данных
detection_boxes = output['detection_boxes'].numpy()[0]  # (1, 100, 4) -> (100, 4)
detection_classes = output['detection_classes'].numpy()[0]  # (1, 100) -> (100,)
detection_scores = output['detection_scores'].numpy()[0]  # (1, 100) -> (100,)
num_detections = int(output['num_detections'].numpy()[0])

# 4. Отображение результатов на изображении с использованием OpenCV
for i in range(num_detections):
    score = detection_scores[i]
    
    # Отображаем только объекты с уверенностью выше порога (например, 0.5)
    if score >= 0.5:
        # Координаты ограничивающей рамки в формате [ymin, xmin, ymax, xmax]
        box = detection_boxes[i]
        ymin, xmin, ymax, xmax = box

        # Поскольку координаты заданы в нормализованном виде (в диапазоне [0, 1]),
        # нужно преобразовать их в координаты изображения
        (im_height, im_width, _) = image_np.shape
        (xmin, xmax, ymin, ymax) = (int(xmin * im_width), int(xmax * im_width), int(ymin * im_height), int(ymax * im_height))

        # Рисуем ограничивающую рамку на изображении
        cv2.rectangle(image_bgr, (xmin, ymin), (xmax, ymax), color=(0, 255, 0), thickness=2)

        # Отображаем класс и оценку уверенности над рамкой
        label = f'Class: {int(detection_classes[i])}, Score: {score:.2f}'
        cv2.putText(image_bgr, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# 5. Отображение изображения с найденными объектами
cv2.imshow('Detected Objects', image_bgr)
cv2.waitKey(0)
cv2.destroyAllWindows()
